## Deliverable 2. Create a Customer Travel Destinations Map.

In [31]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import pprint
from pylab import savefig

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key = g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current_Description
0,0,Hamilton,US,39.1834,-84.5333,80.78,49,100,1.01,overcast clouds
1,1,Narsaq,GL,60.9167,-46.0500,34.02,45,0,3.51,clear sky
2,2,Seryshevo,RU,51.0933,128.3786,34.34,91,72,5.93,broken clouds
3,3,Panguna,PG,-6.3164,155.4848,70.00,95,99,2.17,light rain
4,4,Orange Cove,US,36.6244,-119.3137,86.85,24,1,4.61,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temprature you are okay with? "))
max_temp = float(input("What is the maximum temprature you are okay with? "))


What is the minimum temprature you are okay with? 50
What is the maximum temprature you are okay with? 70


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
perfered_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp)&(city_data_df["Max Temp"]>= min_temp)]

perfered_city_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current_Description
3,3,Panguna,PG,-6.3164,155.4848,70.00,95,99,2.17,light rain
5,5,Cape Town,ZA,-33.9258,18.4232,61.27,76,0,9.46,clear sky
7,7,Nemuro,JP,43.3236,145.5750,61.23,94,81,6.55,broken clouds
9,9,Margate,GB,51.3813,1.3862,61.30,77,100,5.01,overcast clouds
10,10,Jinchang,CN,38.4953,102.1739,61.65,25,31,7.47,scattered clouds
15,15,Airai,TL,-8.9266,125.4092,59.74,57,69,2.48,broken clouds
24,24,Klaksvik,FO,62.2266,-6.5890,52.72,92,99,23.71,overcast clouds
25,25,Bredasdorp,ZA,-34.5322,20.0403,55.27,97,63,6.04,broken clouds
33,33,Mataura,NZ,-46.1927,168.8643,50.49,76,100,4.16,overcast clouds
38,38,Busselton,AU,-33.6500,115.3333,61.05,89,50,17.13,light rain


In [10]:
# 4a. Determine if there are any empty rows.
perfered_city_df.count()


City_ID                200
City                   200
Country                195
Lat                    200
Lng                    200
Max Temp               200
Humidity               200
Cloudiness             200
Wind Speed             200
Current_Description    200
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = perfered_city_df.dropna()
clean_df.count()


City_ID                195
City                   195
Country                195
Lat                    195
Lng                    195
Max Temp               195
Humidity               195
Cloudiness             195
Wind Speed             195
Current_Description    195
dtype: int64

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current_Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current_Description,Lat,Lng,Hotel Name
3,Panguna,PG,70.00,light rain,-6.3164,155.4848,
5,Cape Town,ZA,61.27,clear sky,-33.9258,18.4232,
7,Nemuro,JP,61.23,broken clouds,43.3236,145.5750,
9,Margate,GB,61.30,overcast clouds,51.3813,1.3862,
10,Jinchang,CN,61.65,scattered clouds,38.4953,102.1739,
15,Airai,TL,59.74,broken clouds,-8.9266,125.4092,
24,Klaksvik,FO,52.72,overcast clouds,62.2266,-6.5890,
25,Bredasdorp,ZA,55.27,broken clouds,-34.5322,20.0403,
33,Mataura,NZ,50.49,overcast clouds,-46.1927,168.8643,
38,Busselton,AU,61.05,light rain,-33.6500,115.3333,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
     # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [37]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()
hotel_df.head(10)


,City,Country,Max Temp,Current_Description,Lat,Lng,Hotel Name
3,Panguna,PG,70.00,light rain,-6.3164,155.4848,
5,Cape Town,ZA,61.27,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
7,Nemuro,JP,61.23,broken clouds,43.3236,145.5750,Hotel Nemuro Kaiyoutei
9,Margate,GB,61.30,overcast clouds,51.3813,1.3862,Premier Inn Margate hotel
10,Jinchang,CN,61.65,scattered clouds,38.4953,102.1739,Mingshihui Jinchang International Hotel
15,Airai,TL,59.74,broken clouds,-8.9266,125.4092,Juvinal Martin
24,Klaksvik,FO,52.72,overcast clouds,62.2266,-6.5890,Airbnb rooms
25,Bredasdorp,ZA,55.27,broken clouds,-34.5322,20.0403,Bredasdorp Country Manor
33,Mataura,NZ,50.49,overcast clouds,-46.1927,168.8643,Ellie's Villa
38,Busselton,AU,61.05,light rain,-33.6500,115.3333,Observatory Guest House


In [40]:
# 8a. Create the output File (CSV)
clean_hotel_df = hotel_df.copy()
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df.dropna

<bound method DataFrame.dropna of              City Country  Max Temp Current_Description      Lat       Lng  \
3         Panguna      PG     70.00          light rain  -6.3164  155.4848   
5       Cape Town      ZA     61.27           clear sky -33.9258   18.4232   
7          Nemuro      JP     61.23       broken clouds  43.3236  145.5750   
9         Margate      GB     61.30     overcast clouds  51.3813    1.3862   
10       Jinchang      CN     61.65    scattered clouds  38.4953  102.1739   
..            ...     ...       ...                 ...      ...       ...   
661    Portsmouth      GB     61.00     overcast clouds  50.7990   -1.0913   
666      Imbituba      BR     65.50     overcast clouds -28.2400  -48.6703   
672  Simeonovgrad      BG     56.32     overcast clouds  42.0333   25.8333   
674           Boo      SE     53.76     overcast clouds  59.3333   18.2833   
676        Kijang      KR     66.22           clear sky  35.2442  129.2139   

                             

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current_Description</dt><dd>{Current_Description}</dd> and <dd>{Max Temp} F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

#Save figure ( a manual screen shot????)
savefig("../Vacation_Search/WeatherPy_vacation_map.png")

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>